---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [97]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [98]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [99]:
def histgram(G):
    degrees=G.degree()
    degree_values=sorted(set(degrees.values()))
    #degree_values is a set of unique value of degrees
    #degrees.values() capture all the values that in degrees, can have repited values
    #count each unique values's repiteness
    hist=[list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
    return hist

def graph_identification():
    hist=[len(histgram(G)) for G in P1_Graphs]
    clustering=[nx.average_clustering(G) for G in P1_Graphs]
    path=[nx.average_shortest_path_length(G) for G in P1_Graphs]
    #creating a list of tuples for each group indicating their length of degree values
    #average cluster and average shortest path
    combine=[]
    for i in range(5):
        combine.append(hist[i])
        combine.append(clustering[i])
        combine.append(path[i])
    group=list(zip(combine[0::3],combine[1::3],combine[2::3]))
    results=[]
    for G in group:
        #more choices of degree values
        if G[0]>10:
            results.append('PA')
            #high clustering 
        elif G[1]>0.1:
            results.append('SW_L')
            #small clustering but not follow the power rule of  degree distribution
        else:
            results.append('SW_H')
        
        
    return results
graph_identification()


['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [100]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [101]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
def salary_predictions():
    
    # Your Code Here
    #get the basic information from network
    df=pd.DataFrame(index=G.nodes())
    df['Department']=pd.Series(nx.get_node_attributes(G,'Department'))
    df['Salary']=pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
#creating features
    df['clustering']=pd.Series(nx.clustering(G))
    df['degree']=pd.Series(G.degree())
    df['degree_cen']=pd.Series(nx.degree_centrality(G))
    df['closeness']=pd.Series(nx.closeness_centrality(G,normalized=True))
    df['betweeness']=pd.Series(nx.betweenness_centrality(G,normalized=True,endpoints=False))
    df['pr']=pd.Series(nx.pagerank(G))
#train data set are the nodes with data in salary, others are test data
    df_train=df.dropna()
    df_null=df.isnull()
    df_test=df[df_null['Salary']==True]
    del df_test['Salary']
    df_test
#divide train/test, features/target
    features = ['clustering', 'degree', 'degree_cen', 'closeness', 'betweeness', 'pr','Department']
    X_train=df_train[features]
    X_test=df_test[features]
    y_train=df_train['Salary']
#scaler features
    scaler=MinMaxScaler()
    X_train_scale=scaler.fit_transform(X_train)
    X_test_scale=scaler.fit_transform(X_test)
    #train it with MLP
    clf=MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scale,y_train)
    y_test_proba=clf.predict_proba(X_test_scale)[:,1]
    
    return pd.Series(y_test_proba,X_test.index) # Your Answer Here
salary_predictions()

1       0.373408
2       0.969000
5       0.999863
8       0.350458
14      0.805154
18      0.579049
27      0.720991
30      0.773953
31      0.455403
34      0.309162
37      0.250859
40      0.570420
45      0.500010
54      0.633222
55      0.645400
60      0.489982
62      0.999897
65      0.971275
77      0.156896
79      0.340698
97      0.231199
101     0.156267
103     0.549820
108     0.352342
113     0.905913
122     0.090293
141     0.900412
142     0.997557
144     0.196526
145     0.900957
          ...   
913     0.106342
914     0.166297
915     0.004724
918     0.283145
923     0.051691
926     0.179009
931     0.121348
934     0.030668
939     0.016517
944     0.005730
945     0.056808
947     0.233884
950     0.406778
951     0.066860
953     0.054456
959     0.005915
962     0.005451
963     0.410651
968     0.162520
969     0.148503
974     0.133192
984     0.027200
987     0.209943
989     0.151553
991     0.214990
992     0.007933
994     0.007038
996     0.0054

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [102]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [128]:
def new_connections_predictions():
    
    # Your Code Here
    #creating features with new dataframe
    common_neigh=[(e[0],e[1],len(list(nx.common_neighbors(G,e[0],e[1])))) for e in nx.non_edges(G)]
    df=pd.DataFrame(index=[(e[0],e[1]) for e in common_neigh])
    df['common_neigh']=[e[2] for e in common_neigh]
    #finish creating basic measurements
    jar_neigh=list(nx.jaccard_coefficient(G))
    re_allocation=list(nx.resource_allocation_index(G))
    adamic_adar=list(nx.adamic_adar_index(G))
    pref_attachment=list(nx.preferential_attachment(G))
    df['jar_neigh']=[e[2] for e in jar_neigh]
    df['re_allocation']=[e[2] for e in re_allocation]
    df['adamic_adar']=[e[2] for e in adamic_adar]
    df['pref_attachment']=[e[2] for e in pref_attachment]
    #creating features that require community information
    for n in G.nodes():
        G.node[n]['community']=G.node[n]['Department']
    cn=list(nx.cn_soundarajan_hopcroft(G))
    ra=list(nx.ra_index_soundarajan_hopcroft(G))
    df['cn']=[e[2] for e in cn]
    df['ra']=[e[2] for e in ra]
    df['cn']=df['cn'].fillna(value=0)
    #merge features with target
    results=future_connections.join(df,how='outer')
    #creating train and test dataset
    df_train=results.dropna(subset=['Future Connection'])
    df_test=results[pd.isnull(results['Future Connection'])]
    features=['common_neigh','jar_neigh','re_allocation','adamic_adar','pref_attachment','cn','ra']
    X_train=df_train[features]
    X_test=df_test[features]
    y_train=df_train['Future Connection']
    #scaler features
    scaler=MinMaxScaler()
    X_train_scale=scaler.fit_transform(X_train)
    X_test_scale=scaler.fit_transform(X_test)
    #train it with MLP
    clf=MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scale,y_train)
    y_test_proba=clf.predict_proba(X_test_scale)[:,1]

    
    
    return pd.Series(y_test_proba,X_test.index)# Your Answer Here
new_connections_predictions()

(0, 9)          0.040209
(0, 19)         0.133288
(0, 20)         0.593584
(0, 35)         0.016266
(0, 38)         0.008303
(0, 42)         0.967720
(0, 43)         0.021891
(0, 50)         0.009797
(0, 53)         0.821402
(0, 54)         0.075776
(0, 62)         0.832754
(0, 63)         0.095369
(0, 69)         0.032708
(0, 72)         0.010296
(0, 83)         0.478137
(0, 90)         0.041965
(0, 91)         0.019302
(0, 95)         0.278922
(0, 100)        0.039567
(0, 106)        0.997184
(0, 108)        0.017913
(0, 109)        0.010467
(0, 110)        0.010127
(0, 118)        0.009019
(0, 122)        0.043083
(0, 131)        0.071439
(0, 133)        0.958716
(0, 140)        0.154069
(0, 149)        0.325497
(0, 151)        0.008349
                  ...   
(988, 989)      0.010749
(988, 996)      0.010757
(988, 997)      0.068057
(988, 1000)     0.010743
(988, 1002)     0.010757
(989, 994)      0.010754
(989, 996)      0.010754
(989, 1003)     0.010754
(989, 1004)     0.010754
